## Build the data transformation application

The Production Centre uses Docker images and deploys them in a Cloud based environment for large scale production campaigns.

This step will trigger the build of a Docker image containing the data transformation application created in the previous steps 

We will invoke components of the Ellip Platform Continuous Integration.

The input for trigering the build of Docker image containing the data transformation application uses the pom, an XML file that contains information about the data transformation application and configuration details used by Maven to build the Docker image.

The goal of this Jupyter Notebook is to build of the data transformation application Docker image.

* Define the data transformation application information

In [27]:
app = dict([('artifact_id', 'dcs-s2-ndvi-boa'),
            ('repository', 'Github Organizations'),
            ('community', 'ec-ecopotential')])

* Set the authentication information:

In [12]:
import getpass 

username = raw_input("What is your Jenkins username? ")
api_key = getpass.getpass('And your Jenkins API key:')

What is your Jenkins username? fbrito
And your Jenkins API key:········


* First the import of the required Python libraries:

In [13]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService


* The class with a constructor taking as parameter the path to the pom file and methods to retrieve the version, artifact id and community. This information is used to create the build request



* Create two simple functions to submit the build request and to monitor it:

In [20]:
def submit_build_job(server, job, sleepSecs=3):
    
    last_build = server.get_job_info(job)['lastCompletedBuild']['number']
    
    server.build_job(job, {'RELEASE': 'true'}) 
    
    while server.get_job_info(job)['lastBuild']['number'] == last_build:
        
        time.sleep(sleepSecs)
        
    return server.get_job_info(job)['lastBuild']['number']

In [21]:
def monitor_build_job(server, job, sleepSecs=15):
    
    last_build_number = server.get_job_info(job)['lastBuild']['number']
    
    while server.get_build_info(job, last_build_number)['building']:
        
        time.sleep(sleepSecs)
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'SUCCESS':
        
        return True
    
    if str(server.get_build_info(job, last_build_number)['result']) == 'FAILURE':
        
        return False

* Establish a connection with the build component of the Ellip Platform Continuous Integration:

In [22]:
server = jenkins.Jenkins(url='https://build.terradue.com',
                        username=username,
                        password=api_key)

* Create the build request path:

In [23]:
job = 'communities/%s/%s/%s/docker' % (app['repository'], 
                                      app['community'], 
                                      app['artifact_id'])

In [24]:
print(job)

communities/Github Organizations/ec-ecopotential/dcs-s2-ndvi-boa/docker


* Submit the build job:

In [25]:
submit_build_job(server, job)

37

* Monitor the build request:

In [26]:
if monitor_build_job(server, job):
    print 'Job build completed'
else:
    print 'Job build failed'

Job build completed
